<a href="https://colab.research.google.com/github/2303a52506/Generative-AI-2025/blob/main/GAI_PW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas scikit-learn tensorflow pyswarms openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pyswarms as ps
import os

# Optional: suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Load and clean the data
df_raw = pd.read_excel("Data_Solar Radiation.xlsx", sheet_name="Sheet1")
df = df_raw[1:].copy()
df.columns = df_raw.iloc[0]
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.reset_index(drop=True, inplace=True)

# Convert numerical columns
for col in df.columns[2:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(inplace=True)  # Clean after conversion

# Step 2: Prepare features and target
X = df.drop(columns=['From Date', 'To Date', 'SR'])
y = df['SR'].astype(float)

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 3: Define ANN model builder
def create_model(n_neurons, lr):
    model = Sequential()
    model.add(tf.keras.Input(shape=(X_train.shape[1],)))
    model.add(Dense(int(n_neurons), activation='relu'))
    model.add(Dense(int(n_neurons / 2), activation='relu'))
    model.add(Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Step 4: Fitness function for PSO
def fitness_function(params):
    n_particles = params.shape[0]
    losses = []
    for i in range(n_particles):
        n_neurons = int(params[i][0])
        lr = params[i][1]
        model = create_model(n_neurons, lr)
        model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0)
        predictions = model.predict(X_test, verbose=0)
        mse = mean_squared_error(y_test, predictions)
        losses.append(mse)
    return np.array(losses)

# Step 5: Set PSO options and run optimization
bounds = ([10, 0.0001], [100, 0.1])  # [neurons, learning_rate]
optimizer = ps.single.GlobalBestPSO(
    n_particles=5,
    dimensions=2,
    options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
    bounds=bounds
)

best_cost, best_pos = optimizer.optimize(fitness_function, iters=10)

# Step 6: Train final ANN with best hyperparameters
best_neurons = int(best_pos[0])
best_lr = best_pos[1]

final_model = create_model(best_neurons, best_lr)
final_model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1)

# Step 7: Evaluate model
y_pred = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n🔧 Best Hyperparameters found by PSO:")
print(f"   Neurons: {best_neurons}")
print(f"   Learning Rate: {best_lr:.5f}")

print(f"\n📊 Final Model Evaluation:")
print(f"   Test MSE: {mse:.4f}")
print(f"   R2 Score: {r2:.4f}")


2025-04-05 12:25:15,600 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|10/10, best_cost=363
2025-04-05 12:32:43,084 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 362.5050363855898, best pos: [6.65619596e+01 1.09725879e-03]


Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 8689.8271
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7510.3335
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3637.5220
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 913.8368
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 625.7454
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 594.3759
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 618.2050
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 619.3616
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 611.9681
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 549.4329
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 475.3342
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 449.1726
Epoch 13/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 753.6713
Epoch 14/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 510.0596
Epoch 15/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 